In [ ]:
import sys
import time
import logging
sys.path.append("d:/no-backup/oac-contrail-development/oac/")
import openairclim as oac

# set config file
file_name = "hist.toml"
file_name_old = "hist_old.toml"
file_name_ac = "hist_old_ac.toml"

In [ ]:
# configure the logger
logging.basicConfig(
    format="%(module)s ln. %(lineno)d in %(funcName)s %(levelname)s: %(message)s",
    level=logging.INFO,
    handlers=[
        logging.FileHandler("debug.log", mode="w"),
        logging.StreamHandler(sys.stdout),
    ],
)

# Read the config file
config = oac.get_config(file_name)
config_old = oac.get_config(file_name_old)
config_ac = oac.get_config(file_name_ac)
full_run = config["output"]["full_run"]
output_conc = config["output"]["concentrations"]


inv_species = config["species"]["inv"]
# out_species = config["species"]["out"]
species_0d, species_2d, species_cont, species_sub = (
    oac.classify_species(config)
)
# Read emission inventories
inv_dict = oac.open_inventories(config)
# Adjust emission inventories to given time evolution
inv_dict = oac.adjust_inventories(config, inv_dict)

# Emissions in Tg, each species
_inv_years, emis_dict = oac.get_emissions(inv_dict, inv_species)
_time_range, emis_interp_dict = oac.apply_evolution(
    config, emis_dict, inv_dict, inventories_adjusted=True
)

# load contrail data
ds_cont = oac.open_netcdf_from_config(
    config, "responses", ["cont"], "resp"
)["cont"]
ds_cont_old = oac.open_netcdf_from_config(
    config_old, "responses", ["cont"], "resp"
)["cont"]
ds_cont_ac = oac.open_netcdf_from_config(
    config_ac, "responses", ["cont"], "resp"
)["cont"]

# load base inventories if rel_to_base is TRUE
if config["inventories"]["rel_to_base"]:
    base_inv_dict = oac.open_inventories(config, base=True)
else:
    base_inv_dict = {}

# check contrail input
oac.check_cont_input(config, ds_cont, inv_dict, base_inv_dict)

# get contrail grid
cont_grid = oac.get_cont_grid(ds_cont)
cont_grid_ac = oac.get_cont_grid(ds_cont_ac)

# if necessary, augment base_inv_dict with years in inv_dict not
# present in base_inv_dict
base_inv_dict = oac.interp_base_inv_dict(
    inv_dict, base_inv_dict, ["distance"], cont_grid
)

# Calculate new CFDD and cccov
cfdd_dict = oac.calc_cfdd(config, inv_dict, ds_cont, cont_grid)
cccov_dict = oac.calc_cccov(config, cfdd_dict, ds_cont, cont_grid)
cccov_tot_dict = oac.calc_cccov_tot(config, cccov_dict, cont_grid)
rf_cont_dict = oac.calc_cont_rf(config, cccov_tot_dict, inv_dict, cont_grid)

# Calculate old CFDD and cccov
cfdd_dict_old = oac.calc_cfdd(config_old, inv_dict, ds_cont_old, cont_grid)
cccov_dict_old = oac.calc_cccov(config_old, cfdd_dict_old, ds_cont_old, cont_grid)
cccov_tot_dict_old = oac.calc_cccov_tot(config_old, cccov_dict_old, cont_grid)
rf_cont_dict_old = oac.calc_cont_rf(config_old, cccov_tot_dict_old, inv_dict, cont_grid)

# Calculate AirClim CFDD and cccov
cfdd_dict_ac = oac.calc_cfdd(config_ac, inv_dict, ds_cont_ac, cont_grid_ac)
cccov_dict_ac = oac.calc_cccov(config_ac, cfdd_dict_ac, ds_cont_ac, cont_grid_ac)
cccov_tot_dict_ac = oac.calc_cccov_tot(config_ac, cccov_dict_ac, cont_grid_ac)
rf_cont_dict_ac = oac.calc_cont_rf(config_ac, cccov_tot_dict_ac, inv_dict, cont_grid_ac)

In [ ]:
import xarray as xr
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

year = 2050

ds_cccov = xr.Dataset(
    data_vars={
        "cccov_new": (("lat", "lon"), cccov_dict[year]),
        "cccov_old": (("lat", "lon"), cccov_dict_old[year]),
        "cccov_ac": (("lat", "lon"), cccov_dict_ac[year]),
    },
    coords={
        "lat": (("lat"), cont_grid[1]),
        "lon": (("lon"), cont_grid[0])
    }
)

vmax = 0.01

fig = plt.figure(dpi=300)
ax0 = fig.add_subplot(311, projection=ccrs.PlateCarree())
ax0.coastlines()
ds_cccov.cccov_new.plot(axes=ax0, transform=ccrs.PlateCarree(), vmin=0.0, vmax=vmax)

ax1 = fig.add_subplot(312, projection=ccrs.PlateCarree())
ax1.coastlines()
ds_cccov.cccov_old.plot(axes=ax1, transform=ccrs.PlateCarree(), vmin=0.0, vmax=vmax)

ax2 = fig.add_subplot(313, projection=ccrs.PlateCarree())
ax2.coastlines()
ds_cccov.cccov_ac.plot(axes=ax2, transform=ccrs.PlateCarree(), vmin=0.0, vmax=vmax)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(_time_range, rf_cont_dict["cont"], label="New ISS")
ax.scatter(_time_range, rf_cont_dict_old["cont"], label="Old ISS")
ax.scatter(_time_range, rf_cont_dict_ac["cont"], label="AirClim")
ax.legend(loc="best")